## CERES post-processing

**Inputs:**
* CERES output file (a score for each gene in each cell line) - from `run_CERES.R`
* Reference (non-)essentials from DepMap portal
* Guides per gene from `2_filter_multi_target_guides`
* DepMap gene scores

**Outputs:**
* Cleaned up and filtered gene scores file to use for the rest of the analysis
* Dropped genes (annotated among all screened genes)

For the output, use the same format as the table from DepMap.  
* Genes as column headings
* Cell lines as rows

Post-processing steps:
1. Filter out genes that were not targeted by enough guides (less than 3).    
2. Rescale scores to the reference essentials / non-essentials.
3. Update genes (columns) to Entrez IDs.
4. Filter out non-protein coding genes

In [59]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

get_data_path = lambda folders, fname: os.path.normpath(os.environ['3RD_PARTY_DIR']+'/'+'/'.join(folders) +'/'+fname)
get_local_data_path = lambda folders, fname: os.path.normpath('../data/' +'/'.join(folders) +'/'+ fname)

# Input from running CERES / sgRNA mapping
file_ceres_unscaled = get_local_data_path(['processed', 'depmap19Q1'], 'ceres_output_11_07_19.csv')
file_guides_per_gene = get_local_data_path(['processed', 'depmap19Q1'], 'guides_per_gene_26_05_20.csv')

# Inputs from DepMap
# 20Q2 positive controls: intersection of Hart (2015) and Blomen (2014)
file_ref_essentials = get_data_path(['depmap', '19Q1'], 'essential_genes.txt')
file_ref_nonessentials = get_data_path(['depmap', '19Q1'], 'nonessential_genes.txt')
file_depmap_scores = get_data_path(['depmap', '19Q1'], 'gene_effect.csv')

# Other input
file_id_map = get_data_path(['HGNC'] , 'non_alt_loci_set_01_02_19.txt')

# Output
file_gene_scores = get_local_data_path(['processed', 'depmap19Q1'], 'gene_scores_11_07_19.csv')
file_dropped_genes = get_local_data_path(['processed', 'depmap19Q1'], 'droppped_genes_11_07_19.csv')

### Filter & normalize CERES output

In [15]:
scores_raw = pd.read_csv(file_ceres_unscaled, index_col=0)
scores = scores_raw.T
scores = scores.dropna(axis=1, how='all') # Drop columns (genes) where all values are NaN
scores.index = scores.index.str.replace('.','-')
print('Num genes:', scores.shape[1])
print('Num cell lines:', scores.shape[0])
scores[:2]

Num genes: 17269
Num cell lines: 558


,SHOC2,NDUFA12,SDAD1,FAM98A,ZNF253,HIST1H2BF,SYNE2,SPHAR,BATF2,MYSM1,...,OR2L3,LCE1A,GOLGA6B,NUTM2B,ARL1,IFNA5,SRSF10,STEAP1B,MTRNR2L4,PSG9
ACH-000601,-0.117301,-0.082887,-0.755706,-0.072048,-0.293705,-0.098842,-0.012390,-0.014636,-0.118574,0.053898,...,0.105195,-0.160834,-0.363589,0.134118,-0.153969,-0.018077,-0.437614,0.026585,0.199271,-0.054447
ACH-000496,-0.299054,-0.069327,-0.524179,0.009497,-0.415214,-0.026524,0.023207,0.144555,-0.275711,-0.134776,...,0.196956,-0.011123,-0.383300,0.116085,0.156371,0.074223,-0.474701,0.052686,0.274772,-0.051095


#### 1. Drop genes targetted by too few guides (< 3)

In [34]:
# Drop genes that were targetted by too few guides (less than 3)
# Some genes that I expect to be included might not be there if there was no copy number data for them.
guides_per_gene = pd.read_csv(file_guides_per_gene, index_col=0)
guides_per_gene = guides_per_gene.rename(columns={'ccds_symbol':'symbol'})
print('Genes in guide-gene map:', guides_per_gene.symbol.nunique())
display(guides_per_gene[:2])

print('Genes in CERES output that are not in my guide-per-gene map:')
display(scores.loc[:, ~scores.columns.isin(guides_per_gene.symbol)].columns)

# Check raw scores
print('Genes in my guide-per-gene map that are not in CERES output:')
print(guides_per_gene[~guides_per_gene.symbol.isin(scores_raw.index)].symbol.values)

# Filter guides per gene down to genes that are in CERES output (additional genes dropped are due to NaN values)
guides_per_gene = guides_per_gene[guides_per_gene.symbol.isin(scores.columns)]
print('Filtered guide-per-gene map:', guides_per_gene.shape[0])

Genes in guide-gene map: 17270


,symbol,entrez_id,guides_per_gene
0,A1BG,1,4
1,A1CF,29974,4


Genes in CERES output that are not in my guide-per-gene map:


Index([], dtype='object')

Genes in my guide-per-gene map that are not in CERES output:
['C14orf178']
Filtered guide-per-gene map: 17269


In [36]:
# Drop genes that were targetted by too few guides (less than 3)
print('Genes w/ 2 guides:', guides_per_gene[guides_per_gene.guides_per_gene == 2].shape[0])
print('Genes w/ 1 guide:', guides_per_gene[guides_per_gene.guides_per_gene == 1].shape[0])
scores_filtered = scores.loc[:, scores.columns.isin(guides_per_gene[guides_per_gene.guides_per_gene >= 3].symbol)]
print('Num genes after filtering for too few guides:', scores_filtered.shape[1],'/', scores.shape[1])

Genes w/ 2 guides: 374
Genes w/ 1 guide: 257
Num genes after filtering for too few guides: 16638 / 17269


#### 2. Normalize CERES gene scores to reference essential/non-essential genes from DepMap

In [37]:
# Normalize CERES gene scores, per cell line, according to Hart essential and non-essential genes

def norm(scores, reference_essential, reference_non_essential):
    normed = ((scores - scores[scores.index.isin(reference_non_essential)].median()) / 
              (scores[scores.index.isin(reference_non_essential)].median() -
               scores[scores.index.isin(reference_essential)].median()))
    return normed

# Get the reference essential and non-essentials (downloaded from DepMap)
get_gene_name = lambda x: re.search('([\w-]+)\s\(\w+\)', x).group(1)
reference_essential = pd.read_csv(file_ref_essentials).gene.apply(get_gene_name)
reference_non_essential = pd.read_csv(file_ref_nonessentials).gene.apply(get_gene_name)

# Normalize scores
scores_normed = scores_filtered.apply(lambda x: norm(x, reference_essential, reference_non_essential), axis=1)

# Verify normalization
assert(np.median(scores_normed.loc[:,scores_normed.columns.isin(reference_essential)].values.flatten()) == -1)
assert(np.round(np.median(scores_normed.loc[:,scores_normed.columns.isin(reference_non_essential)].values.flatten()), 15) == 0)
scores_normed[:1]

,SHOC2,NDUFA12,SDAD1,FAM98A,SYNE2,BATF2,MYSM1,EIF2B1,FBXO21,SLC25A24,...,PCDHGB3,YY1AP1,NUDT22,RAB5B,G6PC2,SERPINB10,CGNL1,SLC35D2,ZNF439,ARL1
ACH-000601,-0.117838,-0.083374,-0.75716,-0.07252,-0.012776,-0.119112,0.053607,-1.092377,-0.234541,-0.018627,...,-0.107285,-0.096821,-0.092369,0.004617,0.010656,-0.008256,0.003173,-0.062839,-0.016976,-0.154559


In [38]:
# Check if all entrez ids are in the HGNC gene id map
id_map = pd.read_csv(file_id_map, sep='\t')[['locus_type', 'entrez_id']]
id_map = id_map.dropna(subset=['entrez_id']).astype({'entrez_id':'int'}).astype({'entrez_id':'str'})
entrez_ids_to_discard = guides_per_gene[~guides_per_gene.entrez_id.isin(id_map.entrez_id)].entrez_id.values
print('Genes w/ entrez ids not in HGNC map: ', len(entrez_ids_to_discard), 
       ' -> ', guides_per_gene[~guides_per_gene.entrez_id.isin(id_map.entrez_id)].symbol.values)

# Label genes with entrez id, using guide gene map
rename_map = guides_per_gene[guides_per_gene.entrez_id.isin(id_map.entrez_id)]
scores_renamed = scores_normed.loc[:, scores_normed.columns.isin(rename_map.symbol)]
new_columns = rename_map.set_index('symbol').reindex(scores_renamed.columns).entrez_id.values
assert(scores_renamed.shape[1] == len(new_columns))
scores_renamed.columns = new_columns
scores_renamed.columns = scores_renamed.columns.map(str)
scores_renamed[:2]

Genes w/ entrez ids not in HGNC map:  6  ->  ['C10orf12' 'C10orf131' 'FAM231A' 'MIA2' 'TMEM133' 'UPK3B']


,8036,55967,55153,25940,23224,116071,114803,1967,23014,29957,...,56102,55249,84304,5869,57818,5273,84952,11046,90594,400
ACH-000601,-0.117838,-0.083374,-0.757160,-0.072520,-0.012776,-0.119112,0.053607,-1.092377,-0.234541,-0.018627,...,-0.107285,-0.096821,-0.092369,0.004617,0.010656,-0.008256,0.003173,-0.062839,-0.016976,-0.154559
ACH-000496,-0.301782,-0.067929,-0.530951,0.012311,0.026267,-0.278020,-0.134553,-1.029437,-0.192028,-0.320229,...,-0.033775,0.079118,-0.159384,-0.024264,-0.008371,-0.029345,-0.023362,-0.082542,0.009871,0.161822


In [60]:
# Filter genes to those that are protein coding in HGNC
protein_coding = id_map[id_map.locus_type == 'gene with protein product']
protein_coding_scores = scores_renamed.loc[:, scores_renamed.columns.isin(protein_coding.entrez_id)]
protein_coding_scores = protein_coding_scores.reset_index().rename(columns={'index':'cell_line'})
print('Num protein-coding genes:', protein_coding_scores.shape[1]-1, '/', scores_renamed.shape[1])
protein_coding_scores[:2]

Num protein-coding genes: 16540 / 16632


,cell_line,8036,55967,55153,25940,23224,116071,114803,1967,23014,...,128344,55249,84304,5869,57818,5273,84952,11046,90594,400
0,ACH-000601,-0.117838,-0.083374,-0.757160,-0.072520,-0.012776,-0.119112,0.053607,-1.092377,-0.234541,...,-0.183030,-0.096821,-0.092369,0.004617,0.010656,-0.008256,0.003173,-0.062839,-0.016976,-0.154559
1,ACH-000496,-0.301782,-0.067929,-0.530951,0.012311,0.026267,-0.278020,-0.134553,-1.029437,-0.192028,...,-0.248577,0.079118,-0.159384,-0.024264,-0.008371,-0.029345,-0.023362,-0.082542,0.009871,0.161822


In [61]:
# Export
protein_coding_scores.to_csv(file_gene_scores)

### Record dropped genes (compared to DepMap gene scores file)

In [62]:
# Get the DepMap provided scores (gene_effect.csv) and filter them to my scores
# Original DEPMAP n gene = 17,634 - 6 genes not in HGNC = 17,628 (don't count these 6 genes in analysis)
depmap_scores_all = pd.read_csv(file_depmap_scores, index_col=0)
get_gene_id = lambda x: re.search('[\w-]+\s\((\w+)\)', x).group(1)
depmap_scores_all = depmap_scores_all.rename(columns=get_gene_id)
depmap_scores = depmap_scores_all.loc[:,~depmap_scores_all.columns.isin([str(i) for i in entrez_ids_to_discard])]
depmap_scores[:1]

,1,29974,2,144568,127550,53947,51146,8086,65985,13,...,221302,9183,55055,11130,79364,440590,79699,7791,23140,26009
Broad_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000004,0.115554,0.039815,-0.164159,0.009906,-0.010568,-0.154759,0.236325,-0.335781,0.188261,0.120806,...,-0.164844,-0.148051,-0.082069,-0.363942,0.114826,0.16051,-0.361914,0.184332,0.109661,-0.163671


In [63]:
# Genes that were dropped as a result of guide filtering (not due to non protein-coding)
depmap_genes = pd.DataFrame(depmap_scores.columns, columns=['entrez_id']).astype({'entrez_id':'int'})
depmap_genes = depmap_genes.assign(dropped=~depmap_genes.entrez_id.isin(scores_renamed.columns))
print('Genes dropped as a result of guide filtering:', depmap_genes[depmap_genes.dropped].shape[0])
print('N genes not targeted by any remaining guides:', 
      depmap_genes[~depmap_genes.entrez_id.isin(guides_per_gene.entrez_id)].shape[0])
depmap_genes.to_csv(file_dropped_genes, index=0)
depmap_genes[:1]

Genes dropped as a result of guide filtering: 996
N genes not targeted by any remaining guides: 365


,entrez_id,dropped
0,1,False


### Precision-recall analysis with the DepMap reference essentials

In [53]:
# Reduce scores down to overlap the cell lines and genes in my scores df
depmap_overlap = depmap_scores.loc[depmap_scores.index.isin(scores_renamed.index), 
                                   depmap_scores.columns.isin(scores_renamed.columns)]
my_scores = scores_renamed.loc[scores_renamed.index.isin(depmap_overlap.index),
                               scores_renamed.columns.isin(depmap_overlap.columns)]
print(my_scores.shape, '==', depmap_overlap.shape)

(558, 16632) == (558, 16632)


In [54]:
# Get Hart essentials - the benchmark
get_gene_id = lambda x: re.search('[\w-]+\s\((\w+)\)', x).group(1)
reference_essential = pd.read_csv(file_ref_essentials).gene.apply(get_gene_id)
reference_non_essential = pd.read_csv(file_ref_nonessentials).gene.apply(get_gene_id)
print('Essential reference:', reference_essential.shape[0])
print('Non-essential reference:',reference_non_essential.shape[0])

Essential reference: 212
Non-essential reference: 781


In [55]:
def compute_AUCs(scores, essentials, nonessentials, fdr=0.05):
    AUCs = []
    recall_at_fdr = []
    # Reduce set down to essential and non-essential genes
    scores = scores.loc[:, (scores.columns.isin(essentials)) | (scores.columns.isin(nonessentials))]
    for i in range(0, scores.shape[0]):
        cell_line_scores = scores.iloc[i:i+1, :] #scores for a given cell line
        true_values = cell_line_scores.apply(lambda x: 1 if x.name in essentials else 0).values
        predictions = cell_line_scores.values.flatten()*-1
        precision, recall, thresholds = precision_recall_curve(true_values, predictions)
        PR = pd.DataFrame({'precision': precision, 'recall': recall})
        recall_at_fdr.append(PR[PR.precision >= (1-fdr)].recall.max())
        AUC = auc(recall, precision)
        AUCs.append(AUC)
    return AUCs, recall_at_fdr

In [56]:
depmap_AUC, depmap_recall = compute_AUCs(depmap_overlap, reference_essential.values, reference_non_essential.values)
print('AUC for DepMap gene scores: mean:', np.round(np.mean(depmap_AUC), 4))

AUC for DepMap gene scores: mean: 0.9595


In [57]:
my_AUC, my_recall = compute_AUCs(my_scores, reference_essential.values, reference_non_essential.values)
print('AUC for my reprocessed gene scores: mean:', np.round(np.mean(my_AUC), 4))

AUC for my reprocessed gene scores: mean: 0.9616
